In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

In [3]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

print(np.shape(text_pairs))

(118964, 2)


In [4]:
for _ in range(5):
    print(random.choice(text_pairs))

('Tom denied having done it.', '[start] Tom negó haberlo hecho. [end]')
('Tom is right here.', '[start] Tom está justo aquí. [end]')
('I have a surprise for you.', '[start] Tengo una sorpresa para ti. [end]')
('Tom does a pretty good job of forecasting the weather.', '[start] A Tom se le da bastante bien prever el tiempo. [end]')
('Her skirt is long.', '[start] Su pollera es larga. [end]')


In [5]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [6]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 20 # 20000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

print(np.shape(train_eng_texts))
print(np.shape(train_spa_texts))
print(eng_vectorization)
print(spa_vectorization)

(83276,)
(83276,)


In [7]:
def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

print(train_ds)
print(np.shape(val_ds))

<CacheDataset shapes: ({encoder_inputs: (None, 20), decoder_inputs: (None, 20)}, (None, 20)), types: ({encoder_inputs: tf.int64, decoder_inputs: tf.int64}, tf.int64)>
()


In [30]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [31]:
print(train_ds)

<CacheDataset shapes: ({encoder_inputs: (None, 20), decoder_inputs: (None, 20)}, (None, 20)), types: ({encoder_inputs: tf.int64, decoder_inputs: tf.int64}, tf.int64)>


In [38]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
eng_vocab = eng_vectorization.get_vocabulary()
eng_index_lookup = dict(zip(range(len(eng_vocab)), eng_vocab))
print(inputs["decoder_inputs"])

look_idx = 2

for elem in inputs["encoder_inputs"][look_idx]:
    
    print(elem.numpy(), eng_index_lookup[elem.numpy()])

print()
for elem in inputs["decoder_inputs"][look_idx]:
    
    print(elem.numpy(), spa_index_lookup[elem.numpy()])
    
print()
for elem in targets[look_idx]:
    
    print(elem.numpy(), spa_index_lookup[elem.numpy()])

tf.Tensor(
[[ 2  1 12 ...  0  0  0]
 [ 2  1  6 ...  0  0  0]
 [ 2 10  1 ...  0  0  0]
 ...
 [ 2  1  1 ...  0  0  0]
 [ 2  7  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
2 the
1 [UNK]
8 is
1 [UNK]
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 

2 [start]
10 el
1 [UNK]
12 es
1 [UNK]
3 [end]
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 

10 el
1 [UNK]
12 es
1 [UNK]
3 [end]
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 
0 


In [42]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        
        print(inputs)
        
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):        
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        
        print()
        print(mask)
        print(padding_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [43]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="elu")(x) # activation = "softmax"
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

Tensor("Placeholder:0", shape=(None, None), dtype=int64)
Tensor("Placeholder:0", shape=(None, None), dtype=int64)

Tensor("Placeholder_2:0", shape=(None, None), dtype=bool)
Tensor("transformer_decoder_1/Minimum:0", shape=(None, None, None), dtype=int32)
Tensor("Placeholder:0", shape=(None, None), dtype=int64)

Tensor("model_3/positional_embedding_3/NotEqual:0", shape=(None, None), dtype=bool)
Tensor("model_3/transformer_decoder_1/Minimum:0", shape=(None, None, None), dtype=int32)


In [45]:
epochs = 1  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"], # loss="sparse_categorical_crossentropy"
    run_eagerly = True
)
#transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)
transformer.fit(val_ds, epochs=epochs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_2 (Positio (None, None, 256)    10240       encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_1 (Transfor (None, None, 256)    3155456     positional_embedding_2[0][0]     
________________________________________________________________________________________

  5/279 [..............................] - ETA: 4:38 - loss: 2.6529 - accuracy: 0.3532tf.Tensor(
[[ 6  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 1 18  0 ...  0  0  0]
 ...
 [ 9  1 12 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 6  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[ 2  8  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 [ 2 16  1 ...  0  0  0]
 ...
 [ 2  1  5 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 [ 2  8 17 ...  0  0  0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 .

 11/279 [>.............................] - ETA: 4:17 - loss: 2.8226 - accuracy: 0.4216tf.Tensor(
[[ 1  1  1 ...  0  0  0]
 [ 9  8  1 ...  0  0  0]
 [ 3  1  1 ...  0  0  0]
 ...
 [15  5  1 ...  0  0  0]
 [ 6  1  1 ...  0  0  0]
 [17  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[2 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 [2 1 5 ... 0 0 0]
 ...
 [2 1 1 ... 0 0 0]
 [2 8 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1

 17/279 [>.............................] - ETA: 4:08 - loss: 2.8814 - accuracy: 0.4494tf.Tensor(
[[1 1 0 ... 0 0 0]
 [3 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 4 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[ 2  7 15 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 [ 2  1  3 ...  0  0  0]
 ...
 [ 2  1  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1

 23/279 [=>............................] - ETA: 4:02 - loss: 2.9147 - accuracy: 0.4652tf.Tensor(
[[ 1  1  1 ...  0  0  0]
 [ 1  5  1 ...  0  0  0]
 [ 1  5  1 ...  0  0  0]
 ...
 [ 6  1  1 ...  0  0  0]
 [19  1  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[2 1 1 ... 0 0 0]
 [2 1 6 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 ...
 [2 8 9 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1

...

251/279 [=========================>....] - ETA: 27s - loss: 2.2009 - accuracy: 0.3077tf.Tensor(
[[7 1 1 ... 0 0 0]
 [3 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [6 1 4 ... 0 0 0]
 [3 1 1 ... 0 0 0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[ 2 18  1 ...  0  0  0]
 [ 2  7 14 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 ...
 [ 2  1  1 ...  0  0  0]
 [ 2  8  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 

257/279 [==========================>...] - ETA: 21s - loss: 2.1870 - accuracy: 0.3043tf.Tensor(
[[ 1  1  1 ...  0  0  0]
 [ 6  1  7 ...  0  0  0]
 [17  1  8 ...  0  0  0]
 ...
 [ 3  1  1 ...  0  0  0]
 [ 1  8  7 ...  0  0  0]
 [ 5  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[ 2  1  1 ...  0  0  0]
 [ 2  8  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 ...
 [ 2  1 14 ...  0  0  0]
 [ 2  1 12 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ..

263/279 [===========================>..] - ETA: 15s - loss: 2.1736 - accuracy: 0.3010tf.Tensor(
[[1 1 4 ... 0 0 0]
 [1 5 1 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 0 0 0]
 [6 1 1 ... 0 0 0]
 [1 1 4 ... 0 0 0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[2 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 [2 1 1 ... 1 1 4]
 ...
 [2 4 1 ... 0 0 0]
 [2 8 7 ... 0 0 0]
 [2 1 1 ... 0 0 0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
 

269/279 [===========================>..] - ETA: 9s - loss: 2.1605 - accuracy: 0.2979 tf.Tensor(
[[ 3 16  1 ...  0  0  0]
 [ 6  1 12 ...  0  0  0]
 [ 6  1 16 ...  0  0  0]
 ...
 [ 1  1  1 ...  0  0  0]
 [19  1  1 ...  0  0  0]
 [ 3  1 19 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[2 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 [2 8 7 ... 0 0 0]
 ...
 [2 1 1 ... 0 0 0]
 [2 1 1 ... 0 0 0]
 [2 1 6 ... 0 0 0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 

275/279 [============================>.] - ETA: 3s - loss: 2.1478 - accuracy: 0.2948tf.Tensor(
[[15  5  1 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 [ 6  1  1 ...  0  0  0]
 ...
 [ 1  1  0 ...  0  0  0]
 [ 6  1  1 ...  0  0  0]
 [ 3  1  1 ...  0  0  0]], shape=(64, 20), dtype=int64)
tf.Tensor(
[[ 2  1  1 ...  0  0  0]
 [ 2  1  1 ...  0  0  0]
 [ 2  8  1 ...  0  0  0]
 ...
 [ 2  1  4 ...  0  0  0]
 [ 2  8  7 ...  0  0  0]
 [ 2 14  1 ...  0  0  0]], shape=(64, 20), dtype=int64)

tf.Tensor(
[[ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]], shape=(64, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  [1 1 1 ... 0 0 0]]

 [[1 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 0]
  [1 1 1 ... 0 0 0]
  ...
  [1 1 1 ...

In [47]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        
        print("pred")
        print(predictions)
        
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
        
        return 
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

tf.Tensor([[6 1 1 4 1 1 1 1 1 2 1 1 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)

tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 

pred
tf.Tensor(
[[[-0.99963886 -0.9974384  -0.9993425  -0.9979882  -0.9993322
   -0.9978598  -0.9990609  -0.9973926  -0.9994292  -0.99829096
   -0.9992091  -0.99836    -0.9975804  -0.99730843 -0.9965458
   -0.9959225  -0.9974471  -0.998227   -0.99762416 -0.9995863 ]
  [-0.9996378  -0.99745333 -0.9993414  -0.99798465 -0.9993317
   -0.99785703 -0.99906033 -0.9973911  -0.9994294  -0.9982928
   -0.99920857 -0.99836123 -0.99758506 -0.99730694 -0.99654883
   -0.99591786 -0.9974482  -0.9982253  -0.9976186  -0.99958587]
  [-0.99963754 -0.9974583  -0.9993416  -0.99798346 -0.9993312
   -0.9978552  -0.99905956 -0.9973939  -0.9994293  -0.998293
   -0.9992086  -0.9983584  -0.9975862  -0.99730694 -0.996552
   -0.99591607 -0.99744374 -0.9982259  -0.99761736 -0.9995856 ]
  [-0.99963737 -0.9974625  -0.9993412  -0.9979806  -0.99933153
   -0.9978551  -0.99906075 -0.99739504 -0.999429   -0.99829334
   -0.9992082  -0.9983621  -0.9975893  -0.99730676 -0.99655455
   -0.9959149  -0.997444   -0.99822325 -0.997

tf.Tensor([[6 8 1 1 7 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)

tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

tf.Tensor([[ 1  1  1  1 11  1  1  1  1  0  0  0  0  0  0  0  0  0  0  0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)

tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 

tf.Tensor([[ 1  7  1  1  1 10  1  0  0  0  0  0  0  0  0  0  0  0  0  0]], shape=(1, 20), dtype=int64)
tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)

tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 

tf.Tensor([[2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]], shape=(1, 20), dtype=int64)

tf.Tensor(
[[ True False False False False False False False False False False False
  False False False False False False False False]], shape=(1, 20), dtype=bool)
tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [22]:
test_pairs
input_sentence
translated

'[start] es es es es es es es es es es es es es es es es es es es es'